In [ ]:
import random

import numpy as np
import pandas as pd

import os
from os import listdir
from os.path import join, splitext

import cv2
from PIL import Image

import torch
from torch import nn
from torch.autograd import Variable
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import Compose, ToTensor, ToPILImage

import matplotlib.pyplot as plt

Iterative Crowd Counting Model Libraries

In [ ]:
from icc.data_loaderB import ImageDataLoader
from icc.model_ic_CNN import modelicCNN, retrain_icCNN
from icc.evaluate_icCNN import evaluate_model
from icc import network

GPU Configurations

In [ ]:
# GPU to run on
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="2"

In [ ]:
# Fixing random seed
rand_seed = 26800

np.random.seed(rand_seed)
torch.manual_seed(rand_seed)
torch.cuda.manual_seed(rand_seed)

Hyperparameters to tune

In [ ]:
BATCH_SIZE = 8
MAX_EPOCH = 1000

LR = 0.00001
WEIGHT_DECAY = 0.00001
MOMENTUM = 0.9

Data Loader Configuration

In [ ]:
train_path = 'data/train/images'
train_gt_path = 'data/train/ground_truth_csv'

val_path = 'data/val/images'
val_gt_path = 'data/val/ground_truth_csv'

output_dir = 'logs/model_icCNN/'

In [ ]:
train_data_loader = ImageDataLoader(train_path, 
                                    train_gt_path,
                                    shuffle=True,
                                    gt_downsample=False,
                                    pre_load=True,
                                    sr_mode=True)

val_data_loader = ImageDataLoader(val_path, 
                                  val_gt_path,
                                  shuffle=True,
                                  gt_downsample=False,
                                  pre_load=True,
                                  sr_mode=True)

Utils to read and transform data

In [ ]:
def RandomCrop(Input, Density, h, w, th, tw):
    x1 = random.randint(0, h - th)
    y1 = random.randint(0, w - tw)

    Input = Input[x1:x1 + th, y1:y1 + tw]
    Density = Density.reshape((h, w))[x1:x1 + th, y1:y1 + tw]

    return Input, Density

In [ ]:
def get_training_batch(blob):
    img = blob['data']
    gt_density = blob['gt_density']

    h = img.shape[0]
    w = img.shape[1]
    
    th = int(h/3.0 - ((h/3.0) % 4))
    tw = int(w/3.0 - ((w/3.0) % 4))
    
    Input_HR = torch.zeros(BATCH_SIZE, 3, th, tw)
    GT_Density = torch.zeros(BATCH_SIZE, 1, th, tw)

    for cur_step in range(0, BATCH_SIZE):
        img_crop, gt_density_crop = RandomCrop(img, gt_density, h, w, th, tw)

        Input_HR[cur_step] = ToTensor()(img_crop)
        GT_Density[cur_step] = torch.from_numpy(gt_density_crop)
        
    return Input_HR, GT_Density

Initializing ICC parameters

In [ ]:
# Initializing model
net = modelicCNN()
network.weights_normal_init(net.netDME)
network.weights_normal_init(net.netFCNN)
network.weights_normal_init(net.netGCE)

net.cuda()
net.train()

# Initializing optimizer
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, net.parameters()), lr=LR)
# optimizer = torch.optim.SGD(net.parameters(), lr=LR, weight_decay=WEIGHT_DECAY, momentum=MOMENTUM)

# Initializing loss
n_loss_fn = nn.MSELoss()

Training Networks

In [ ]:
best_maeHR = float('inf') #sys.maxint
best_epochHR = 1

N_Loss = []

In [ ]:
for epoch in range(1, MAX_EPOCH+1):
    
    count = 0
    n_loss = 0
    for blob in train_data_loader:
        optimizer.zero_grad()

        Input_HR, GT_Density = get_training_batch(blob)
        
        Input_HR = Input_HR.cuda()
        GT_Density = GT_Density.cuda()
                
        Density = net(Input_HR)
        
        LossHR = n_loss_fn(Density, GT_Density)
        
        loss = 1000 * LossHR
        loss.backward()        
        optimizer.step()
        
        count += 1
        
        n_loss += loss.data.item()
        print("Training epoch: {}, Batch_Num: {}/{}, N_Loss: {}".format(
            epoch, count, train_data_loader.get_num_samples(), loss.data.item()))
        
        del Input_HR, GT_Density
        
    N_Loss.append(n_loss/count)
    
    if (epoch % 5 == 0):
        maeHR, mseHR = evaluate_model(net, val_data_loader)
        net.train()

        if maeHR < best_maeHR:
            best_maeHR = maeHR
            best_mseHR = mseHR
            best_epochHR = epoch

            network.save_net(os.path.join(output_dir, 'best_icCNN.h5'), net)

        print("EPOCH: %d, MAE_HR: %.1f, MSE_HR: %0.1f" % (epoch, maeHR, mseHR))
        print("BEST MAE_HR: %0.1f, BEST MSE_HR: %0.1f, BEST Epoch: %4.2f" % (best_maeHR, best_mseHR, best_epochHR))

In [ ]:
network.save_net(os.path.join(output_dir, 'last_epoch_icCNN.h5'), net)

In [ ]:
print(N_Loss)